In [4]:
pip install pandas scikit-learn


  Using cached scikit_learn-1.5.0-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.13.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.0-cp312-cp312-win_amd64.whl (10.9 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.13.1-cp312-cp312-win_amd64.whl (45.9 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import (
    GaussianNB,
    MultinomialNB,
    CategoricalNB,
    ComplementNB,
    BernoulliNB,
)
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
# Combine all PDBs into a single dataframe
ring_path = Path("C:/Users/emmal/OneDrive/Desktop/StructuralBioinformatics/PROJECT/features_ring")
dfs = []
for filename in os.listdir(ring_path):
    dfs.append(pd.read_csv(ring_path / filename, sep="\t"))
df = pd.concat(dfs)
df


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\emmal\\OneDrive\\Desktop\\StructuralBioinformatics\\PROJECT\\features_ring\\features_ring'

In [15]:
new_df = pd.DataFrame(df.iloc[:1000000, :])
new_df.dropna(inplace=True)

y = new_df["Interaction"].astype("category")

# Define training features
X = new_df[
    [
        "s_rsa",
        "s_up",
        "s_down",
        "s_phi",
        "s_psi",
        "s_a1",
        "s_a2",
        "s_a3",
        "s_a4",
        "s_a5",
        "t_rsa",
        "t_up",
        "t_down",
        "t_phi",
        "t_psi",
        "t_a1",
        "t_a2",
        "t_a3",
        "t_a4",
        "t_a5",
    ]
]

# Calculate percentiles and transform into categories
X = X.rank(pct=True).round(1).astype("category")

# Split the dataset to define training and testing examples
X_new_train, X_new_test, y_new_train, y_new_test = train_test_split(
    X, y, test_size=0.1, random_state=0
)



NameError: name 'df' is not defined

In [54]:
bonds = ["HBOND", "IONIC", "PICATION", "PIHBOND", "PIPISTACK", "SSBOND", "VDW"]

for bond in bonds:
    print(f"this is new_df {bond}: {len(new_df[new_df["Interaction"] == bond])}")
    print(f"this is df{bond}: {len(df[df["Interaction"] == bond])}\n")

this is new_df HBOND: 557203
this is dfHBOND: 820333

this is new_df IONIC: 18494
this is dfIONIC: 27538

this is new_df PICATION: 4583
this is dfPICATION: 6874

this is new_df PIHBOND: 1127
this is dfPIHBOND: 1669

this is new_df PIPISTACK: 20447
this is dfPIPISTACK: 30117

this is new_df SSBOND: 1126
this is dfSSBOND: 1657

this is new_df VDW: 397020
this is dfVDW: 583192



In [40]:
# Remove all rows with NaN in at least one column
# including rows with missing class (they could be false negatives)
df.dropna(inplace=True)

# Define ground truth values
y = df["Interaction"].astype("category")
y

1       HBOND
3       HBOND
6       HBOND
7         VDW
9       HBOND
        ...  
1030    HBOND
1031      VDW
1032    HBOND
1033    HBOND
1035      VDW
Name: Interaction, Length: 1471380, dtype: category
Categories (7, object): ['HBOND', 'IONIC', 'PICATION', 'PIHBOND', 'PIPISTACK', 'SSBOND', 'VDW']

In [7]:
# Define training features
X = df[
    [
        "s_rsa",
        "s_up",
        "s_down",
        "s_phi",
        "s_psi",
        "s_a1",
        "s_a2",
        "s_a3",
        "s_a4",
        "s_a5",
        "t_rsa",
        "t_up",
        "t_down",
        "t_phi",
        "t_psi",
        "t_a1",
        "t_a2",
        "t_a3",
        "t_a4",
        "t_a5",
    ]
]

# Calculate percentiles and transform into categories
X = X.rank(pct=True).round(1).astype("category")
X 

,s_rsa,s_up,s_down,s_phi,s_psi,s_a1,s_a2,s_a3,s_a4,s_a5,t_rsa,t_up,t_down,t_phi,t_psi,t_a1,t_a2,t_a3,t_a4,t_a5
1,0.1,0.9,0.9,0.6,0.4,0.2,0.2,0.3,0.9,0.3,0.5,0.5,0.5,0.6,0.1,0.4,0.1,0.4,1.0,0.5
3,0.6,0.6,0.0,0.2,0.9,1.0,0.3,0.5,0.2,0.9,0.9,0.1,0.5,0.3,0.7,0.7,0.8,0.6,0.4,0.7
6,0.5,0.5,0.4,0.6,0.1,0.3,0.1,0.4,1.0,0.5,0.8,0.0,0.8,1.0,0.6,0.4,1.0,0.6,0.7,0.9
7,0.5,0.5,0.4,0.6,0.1,0.3,0.1,0.4,1.0,0.5,0.8,0.0,0.8,1.0,0.6,0.4,1.0,0.6,0.7,0.9
9,0.7,0.4,0.5,0.1,0.7,0.2,0.3,0.8,0.2,0.6,0.1,0.6,0.9,0.4,0.4,0.6,0.6,0.1,0.2,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1030,0.5,0.6,0.0,0.0,0.6,1.0,0.3,0.5,0.2,0.9,0.3,0.7,0.2,0.3,1.0,0.5,0.9,0.0,0.7,0.1
1031,0.5,0.6,0.0,0.0,0.6,1.0,0.3,0.5,0.2,0.9,0.3,0.7,0.2,0.3,1.0,0.5,0.9,0.0,0.7,0.1
1032,0.9,0.2,0.4,0.1,0.7,0.8,0.7,0.1,0.3,0.0,0.5,0.2,0.5,1.0,0.6,0.3,0.0,1.0,0.1,0.7
1033,0.6,0.6,0.1,0.5,0.3,0.2,0.3,0.8,0.2,0.6,0.9,0.0,0.8,1.0,0.6,0.4,1.0,0.6,0.7,0.9


In [9]:
# Split the dataset to define training and testing examples
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

### Test different versions of Naive Bayes

In [41]:
nb = GaussianNB()
y_new_pred = nb.fit(X_new_train, y_new_train).predict(X_new_test)
print(
    "Number of mislabeled points out of a total %d points : %d"
    % (X_new_test.shape[0], (y_new_test != y_new_pred).sum())
)

Number of mislabeled points out of a total 100000 points : 49176


In [42]:
nb = MultinomialNB()
y_new_pred = nb.fit(X_new_train, y_new_train).predict(X_new_test)
print(
    "Number of mislabeled points out of a total %d points : %d"
    % (X_new_test.shape[0], (y_new_test != y_new_pred).sum())
)

Number of mislabeled points out of a total 100000 points : 44269


In [43]:
nb = ComplementNB()
y_new_pred = nb.fit(X_new_train, y_new_train).predict(X_new_test)
print(
    "Number of mislabeled points out of a total %d points : %d"
    % (X_new_test.shape[0], (y_new_test != y_new_pred).sum())
)

Number of mislabeled points out of a total 100000 points : 60680


In [44]:
nb = BernoulliNB()
y_new_pred = nb.fit(X_new_train, y_new_train).predict(X_new_test)
print(
    "Number of mislabeled points out of a total %d points : %d"
    % (X_new_test.shape[0], (y_new_test != y_new_pred).sum())
)

Number of mislabeled points out of a total 100000 points : 44009


In [45]:
nb = CategoricalNB()
y_new_pred = nb.fit(X_new_train, y_new_train).predict(X_new_test)
print(
    "Number of mislabeled points out of a total %d points : %d"
    % (X_new_test.shape[0], (y_new_test != y_new_pred).sum())
)

Number of mislabeled points out of a total 100000 points : 43690


# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

# Logistic Regression
lr = LogisticRegression(max_iter=1000, random_state=42)

# Validation
lr.fit(X_new_train, y_new_train)

# Prediction
y_pred_lr = lr.predict(X_new_test)
print("Logistic Regression Classification Report:")

# Accuracy
accuracy = accuracy_score(y_new_test, y_new_pred)
print(f"Accuracy: {accuracy:.2f}")

# Classification report 
print("Classification Report:")
print(classification_report(y_new_test, y_new_pred))
